In [1]:
### GMOSS_python_integration_tests.ipynb
##
## Generate GMOSS spectra towards select healpix pixels to find fastest integration algorithms in python
## Using parameters file in GMOSS_params_allpix.txt
##
## Based on code in GMOSS_8aug17.c
##
## Uses a freq_array defined in section below as the frequency axis

import numpy as np
import matplotlib.pyplot as plt
import math
import csv
import healpy as hp
import aipy as a
import scipy.constants
import scipy.integrate
import scipy.io
import pickle
import datetime

PI = np.pi

FILE_PARAMS = 'GMOSS_params_allpix.txt'


ModuleNotFoundError: No module named 'aipy'

In [ ]:
### Define frequency array
###
freq_array = np.linspace(40.0,230.0,num=191,endpoint=True,dtype=float)  # MHz

print("Number of frequencies: ",len(freq_array))

freq_GHz = freq_array/1000.0
print(" Frequency array ends: ",freq_GHz[0],freq_GHz[1],".....",freq_GHz[-2],freq_GHz[-1]," GHz. ")

In [ ]:
### Define Nested Healpix sky with nside 16 (3072 pixels)
###
NSIDE = 16   # hardcoded to conform to the content of FILE_PARAMS

npix = hp.pixelfunc.nside2npix(NSIDE)
resol = hp.pixelfunc.nside2resol(NSIDE,arcmin=True)
print("Generate spectra at healpix with nside, npix, resol (arcmin): ",NSIDE, npix, resol)

In [ ]:
### Read in the parameter file
###
flag = []
norm_param = []
alpha1_param = []
dalpha_param = []
nubrk_param = []
Tx_param = []
Te_param = []
nut_param = []

with open(FILE_PARAMS) as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        if row != []:
            row = row[0].split()
            
            flag.append(int(row[1]))
            norm_param.append(float(row[2]))
            alpha1_param.append(float(row[3]))
            dalpha_param.append(float(row[4]))
            nubrk_param.append(float(row[5]))
            Tx_param.append(float(row[6]))
            Te_param.append(float(row[7]))
            nut_param.append(float(row[8]))

print("Got parameters in ",len(Tx_param)," pixels.")

In [ ]:
## Iterate over sky pixels, computing the entire spectrum at each sky position

import warnings
warnings.filterwarnings("ignore")

band_definition = [[ 41.430528,  82.861056], \
       [ 57.5424  , 115.0848  ], \
       [ 79.92    , 159.84    ], \
       [111.      , 222.      ]]

GSPAN = 100.0  # defines a factor by which the range of integration is expanded beyond required boundary

q_e = 1.6e-19
Bmag = 1e-9  # Tesla == 10 micro-Gauss
sin_alph = 1.0
m_e = 9.1e-31
cvel = scipy.constants.c
scale_gam_nu = (3.0*q_e*Bmag*sin_alph)/(4.0*PI*m_e*cvel)

# Return value of polynomial
func_poly = lambda p, x    : (np.polyval(p,x))

def modbessik2(u):  # Modified Bessel Fn of second kind 
                    # for non-negative real fractional order for real positive argument
                    # Takes numpy arrays as inputs

    xnu = 5.0/3.0  # fractional order
    uu = np.where ((u<0.01),0.01,u)
    arg = np.reciprocal(uu) # argument
    bk = scipy.special.kve(xnu, arg, out=None)
    xrk = np.divide(bk,np.exp(arg))
    return (np.divide(xrk,np.multiply(uu,uu)))
    
def fofx1float(gama,nu,scale_gam_nu,C1,fmult):  ## takes numpy arrays as inputs
    
    nu_c = np.multiply( np.multiply(gama,gama) , (scale_gam_nu/1e9) )
    x = np.divide(nu,nu_c)
    xu = np.reciprocal(x)
    if np.isscalar(xu) == True: 
        xl = 0.0
        rint =  scipy.integrate.romberg(modbessik2,xl,xu,vec_func=True) 
        
    else: 
        rint = []
        for i in range (len(xu)):
            xl = 0.0
            xu0 = xu[i]
            rint.append(scipy.integrate.romberg(modbessik2,xl,xu0,vec_func=True) )
     
    p1 = -((2*C1) - 3.0)
    integ = np.multiply( np.multiply(fmult,rint) , np.multiply( np.power(gama,p1) , (x) ) )    
    return integ

In [ ]:
spectrum = []

print("Begin iterations at time: ",datetime.datetime.now())

In [ ]:
j = int(input('Enter pixel number: ') or 100) 
    
Tx = 10.0**Tx_param[j]
Te = 10.0**Te_param[j]
nu_t = 10.0**nut_param[j]
alpha1 = 10.0**alpha1_param[j]
    
peak_residual = 100.0

TTOL = 1.48e-3 # 1.48e-3  # tolerances for the romberg integrations
RTOL = 1.48e-6 # 1.48e-6
    
while (peak_residual > 0.001 and RTOL >1.48e-9):
        
    cspect = []

    if flag[j] == 0:  ## the case where alpha2 steeper than alpha1

        fnorm = 10.0**norm_param[j]
        alpha2 = 10.0**dalpha_param[j] + 10.0**alpha1_param[j]
        nu_break = 10.0**nubrk_param[j]
        gama_break = np.sqrt((nu_break)/scale_gam_nu)
        xb = gama_break

        for ii in range (len(freq_GHz)):

            nu = freq_GHz[ii]
            nu_min = nu*1e9/GSPAN
            nu_max = nu*1e9*GSPAN
            gama_min = np.sqrt((nu_min)/scale_gam_nu)
            gama_max = np.sqrt((nu_max)/scale_gam_nu)
            xl = gama_min
            xu = gama_max

            if  xl > xb:
                C1 = alpha2
                fmult =  ((gama_break)**(2*C1-3))
                rint =  scipy.integrate.romberg(fofx1float,xl,xu,\
                                    args=(nu,scale_gam_nu,C1,fmult),tol=TTOL, rtol=RTOL,vec_func=True)  

            elif xu < xb:
                C1 = alpha1
                fmult = ((gama_break)**(2*C1-3))
                rint =  scipy.integrate.romberg(fofx1float,xl,xu,\
                                    args=(nu,scale_gam_nu,C1,fmult),tol=TTOL, rtol=RTOL, vec_func=True) 

            else:
                xu = xb
                C1 = alpha1
                fmult = ((gama_break)**(2*C1-3))
                rint1 =  scipy.integrate.romberg(fofx1float,xl,xu,\
                                    args=(nu,scale_gam_nu,C1,fmult),tol=TTOL, rtol=RTOL,vec_func=True) 
                xl = xb
                xu = gama_max
                C1 = alpha2
                fmult = ((gama_break)**(2*C1-3))
                rint2 =  scipy.integrate.romberg(fofx1float,xl,xu,\
                                        args=(nu,scale_gam_nu,C1,fmult),tol=TTOL, rtol=RTOL,vec_func=True) 
                rint = rint1 + rint2

            extn = np.exp(-1.0*((nu_t/nu)**2.1))
            cspect.append( fnorm*((nu**-2.0)*rint + Tx*(nu**-2.1))*extn + Te*(1.0-extn) )

    else:            ##   If data requires alpha2 flatter than alpha1, 
                         ##   then model as sum of power laws, i.e. steep and flat spectrum sources

        fnorm1 = norm_param[j]
        fnorm2 = nubrk_param[j]
        alpha2 = alpha1 - 10.0**dalpha_param[j]

        for ii in range (len(freq_GHz)):

            nu = freq_GHz[ii]
            extn = np.exp(-1.0*((nu_t/nu)**2.1))
            cspect.append(   fnorm1*( (nu**(-alpha1)) + fnorm2*(nu**(-alpha2)) + \
                                        Tx*(nu**-2.1) )*extn + Te*(1.0 - extn)   )

    cspect_residual = []
    for iband in range (len(band_definition)):
        band_low = band_definition[iband][0]
        band_high = band_definition[iband][1]

        x00 = np.copy(freq_array)
        ilow = np.argmin(np.abs(x00-band_low))
        ihigh = np.argmin(np.abs(x00-band_high))
        xx = (freq_array[ilow:ihigh+1])
        yy = (cspect[ilow:ihigh+1])
        norder = 7
        zz = np.polyfit(np.log10(xx),np.log10(yy), norder)
        yy_fit = 10.0**( np.array(func_poly(zz,np.log10(xx))) )
        yy_res = yy - yy_fit
        cspect_residual = np.append(cspect_residual,yy_res,axis=0)

    peak_residual = np.max( np.abs(cspect_residual))
        #print("Peak residual: ",peak_residual," with RTOL: ",RTOL)
    RTOL = RTOL/10.0
        
    
    print("At pixel: ",j," of ",npix,"Peak residual: ",peak_residual," at time: ",datetime.datetime.now())

    spectrum.append(cspect)

